In [1]:
!pip install fastapi uvicorn nest_asyncio pydantic httpx python-dotenv openai


In [2]:
import os
import re
import json
import threading
import datetime as dt
from typing import List, Optional, Dict, Any

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

try:
    import openai
    OPENAI_AVAILABLE = True
except Exception:
    OPENAI_AVAILABLE = False


In [3]:
class Task(BaseModel):
    id: str
    title: str
    description: str
    duration_days: int
    start_after: Optional[List[str]] = []
    suggested_start_date: Optional[str] = None
    suggested_end_date: Optional[str] = None


class PlanResponse(BaseModel):
    goal: str
    mode: str
    timeframe_days: int
    tasks: List[Task]
    generated_at: str


In [4]:
class Planner:
    """Planner uses OpenAI if available, otherwise falls back to local deterministic planning."""

    def __init__(self, openai_api_key: Optional[str] = None, default_timezone: str = 'UTC'):
        self.openai_api_key = openai_api_key or os.getenv('OPENAI_API_KEY')
        self.default_timezone = default_timezone
        if self.openai_api_key and OPENAI_AVAILABLE:
            openai.api_key = self.openai_api_key
            self.use_openai = True
        else:
            self.use_openai = False

    def extract_timeframe_days(self, goal: str) -> int:
        goal_low = goal.lower()
        m = re.search(r'(?:in|within|by)?\s*(\d+)\s*days', goal_low)
        if m:
            return int(m.group(1))
        m = re.search(r'(?:in|within|by)?\s*(\d+)\s*weeks', goal_low)
        if m:
            return int(m.group(1)) * 7
        m = re.search(r'(?:in|within|by)?\s*(\d+)\s*months', goal_low)
        if m:
            return int(m.group(1)) * 30
        if re.search(r'launch|release|demo|ship|mvp', goal_low):
            return 14
        return 28

    def local_decompose(self, goal: str, timeframe_days: int) -> List[Task]:
        phases = [
            ("Discovery & Requirements", "Gather requirements, scope, and success criteria."),
            ("High-level Plan", "Define milestones, deliverables and acceptance criteria."),
            ("Design / Architecture", "Design UI/UX, data models and system architecture."),
            ("Implementation / Build", "Develop features in prioritized order."),
            ("Testing & QA", "Perform testing and bug fixing."),
            ("Launch / Delivery", "Deploy and monitor post-launch."),
        ]
        alloc = [0.08, 0.07, 0.15, 0.45, 0.15, 0.10]
        base = max(1, timeframe_days)
        tasks: List[Task] = []
        today = dt.date.today()
        cursor = today

        for i, (title, desc) in enumerate(phases):
            dur = max(1, int(round(base * alloc[i])))
            t = Task(
                id=f"T{i+1}",
                title=title,
                description=desc,
                duration_days=dur,
                start_after=[f"T{i}"] if i > 0 else [],
            )
            t.suggested_start_date = cursor.isoformat()
            end = cursor + dt.timedelta(days=t.duration_days - 1)
            t.suggested_end_date = end.isoformat()
            cursor = end + dt.timedelta(days=1)
            tasks.append(t)
        return tasks

    def call_openai(self, goal: str, timeframe_days: int) -> List[Dict[str, Any]]:
        prompt = (
            f"Break down this goal into actionable tasks with durations and dependencies. "
            f"Goal: {goal}\nTimeframe days: {timeframe_days}\n"
            f"Return JSON array with fields: id, title, description, duration_days, start_after (list of ids)."
        )
        try:
            if not self.use_openai:
                raise RuntimeError('OpenAI not configured')
            resp = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You output only JSON arrays of tasks."},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=800,
                temperature=0.3,
            )
            content = resp["choices"][0]["message"]["content"]
            return json.loads(content)
        except Exception as e:
            print("OpenAI planning failed, using local fallback:", e)
            return [t.dict() for t in self.local_decompose(goal, timeframe_days)]

    def generate_plan(self, goal: str, mode: str = 'auto') -> PlanResponse:
        tf_days = self.extract_timeframe_days(goal)
        if mode == 'openai' and self.use_openai:
            raw_tasks = self.call_openai(goal, tf_days)
            tasks = [Task(**r) for r in raw_tasks]
            mode_used = "openai"
        else:
            tasks = self.local_decompose(goal, tf_days)
            mode_used = "local"
        return PlanResponse(
            goal=goal,
            mode=mode_used,
            timeframe_days=tf_days,
            tasks=tasks,
            generated_at=dt.datetime.utcnow().isoformat() + "Z",
        )


In [5]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI(title="Smart Task Planner", version="1.0")
app.add_middleware(
    CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"]
)

planner = Planner()

class GenerateRequest(BaseModel):
    goal: str
    mode: Optional[str] = 'auto'

@app.post("/generate_plan", response_model=PlanResponse)
async def generate_plan(req: GenerateRequest):
    if not req.goal.strip():
        raise HTTPException(status_code=400, detail="Goal is required")
    plan = planner.generate_plan(req.goal, mode=req.mode)
    return plan

@app.get("/health")
async def health():
    return {"status": "ok", "openai_enabled": planner.use_openai}


In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8001, log_level="info")

thread = threading.Thread(target=run_app, daemon=True)
thread.start()


INFO:     Started server process [20032]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:52698 - "POST /generate_plan HTTP/1.1" 200 OK


C:\Users\HP\AppData\Local\Temp\ipykernel_20032\637558352.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  generated_at=dt.datetime.utcnow().isoformat() + "Z",


INFO:     127.0.0.1:52701 - "GET /health HTTP/1.1" 200 OK


In [7]:
import httpx

response = httpx.post(
    "http://127.0.0.1:8001/generate_plan",
    json={"goal": "Launch a product in 2 weeks", "mode": "local"}
)

print("Status:", response.status_code)
print(json.dumps(response.json(), indent=2))


Status: 200
{
  "goal": "Launch a product in 2 weeks",
  "mode": "local",
  "timeframe_days": 14,
  "tasks": [
    {
      "id": "T1",
      "title": "Discovery & Requirements",
      "description": "Gather requirements, scope, and success criteria.",
      "duration_days": 1,
      "start_after": [],
      "suggested_start_date": "2025-10-15",
      "suggested_end_date": "2025-10-15"
    },
    {
      "id": "T2",
      "title": "High-level Plan",
      "description": "Define milestones, deliverables and acceptance criteria.",
      "duration_days": 1,
      "start_after": [
        "T1"
      ],
      "suggested_start_date": "2025-10-16",
      "suggested_end_date": "2025-10-16"
    },
    {
      "id": "T3",
      "title": "Design / Architecture",
      "description": "Design UI/UX, data models and system architecture.",
      "duration_days": 2,
      "start_after": [
        "T2"
      ],
      "suggested_start_date": "2025-10-17",
      "suggested_end_date": "2025-10-18"
    },
 

In [8]:
import httpx
print(httpx.get("http://127.0.0.1:8001/health").json())


{'status': 'ok', 'openai_enabled': False}


In [9]:
# Step 6: Test the Smart Task Planner API inside Jupyter

from fastapi.testclient import TestClient

client = TestClient(app)

# You can change this goal text to anything you want
goal_input = {
    "goal": "Launch a new mobile app in 3 weeks"
}

response = client.post("/generate_plan", json=goal_input)

print("Status Code:", response.status_code)
print("\nGenerated Smart Task Plan:\n")
print(response.json())


Status Code: 200

Generated Smart Task Plan:

{'goal': 'Launch a new mobile app in 3 weeks', 'mode': 'local', 'timeframe_days': 21, 'tasks': [{'id': 'T1', 'title': 'Discovery & Requirements', 'description': 'Gather requirements, scope, and success criteria.', 'duration_days': 2, 'start_after': [], 'suggested_start_date': '2025-10-15', 'suggested_end_date': '2025-10-16'}, {'id': 'T2', 'title': 'High-level Plan', 'description': 'Define milestones, deliverables and acceptance criteria.', 'duration_days': 1, 'start_after': ['T1'], 'suggested_start_date': '2025-10-17', 'suggested_end_date': '2025-10-17'}, {'id': 'T3', 'title': 'Design / Architecture', 'description': 'Design UI/UX, data models and system architecture.', 'duration_days': 3, 'start_after': ['T2'], 'suggested_start_date': '2025-10-18', 'suggested_end_date': '2025-10-20'}, {'id': 'T4', 'title': 'Implementation / Build', 'description': 'Develop features in prioritized order.', 'duration_days': 9, 'start_after': ['T3'], 'suggested

C:\Users\HP\AppData\Local\Temp\ipykernel_20032\637558352.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  generated_at=dt.datetime.utcnow().isoformat() + "Z",


In [10]:
# Step 7: Create index.html frontend for Smart Task Planner

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Smart Task Planner</title>
  <style>
    body {
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      background-color: #f3f4f6;
      color: #333;
      margin: 0;
      padding: 2rem;
      display: flex;
      flex-direction: column;
      align-items: center;
    }
    h1 {
      color: #2563eb;
    }
    textarea {
      width: 400px;
      height: 100px;
      padding: 10px;
      border-radius: 8px;
      border: 1px solid #ccc;
      font-size: 1rem;
      margin-bottom: 1rem;
    }
    button {
      background-color: #2563eb;
      color: white;
      border: none;
      padding: 10px 20px;
      border-radius: 8px;
      cursor: pointer;
      font-size: 1rem;
    }
    button:hover {
      background-color: #1e40af;
    }
    #output {
      margin-top: 2rem;
      width: 500px;
      background: white;
      padding: 1rem;
      border-radius: 10px;
      box-shadow: 0 2px 6px rgba(0,0,0,0.1);
    }
    .task-item {
      margin-bottom: 8px;
      border-bottom: 1px dashed #ddd;
      padding-bottom: 6px;
    }
  </style>
</head>
<body>
  <h1>🧠 Smart Task Planner</h1>
  <textarea id="goalInput" placeholder="Enter your goal (e.g., Launch a product in 2 weeks)..."></textarea>
  <br />
  <button onclick="generatePlan()">Generate Plan</button>
  <div id="output"></div>

  <script>
    async function generatePlan() {
      const goalText = document.getElementById('goalInput').value.trim();
      const output = document.getElementById('output');
      output.innerHTML = '<p>⏳ Generating plan...</p>';

      const response = await fetch('/generate_plan', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ goal: goalText })
      });

      const data = await response.json();
      output.innerHTML = '<h2>🗂️ Task Plan:</h2>';
      data.tasks.forEach((t, i) => {
        output.innerHTML += `
          <div class="task-item">
            <b>${i + 1}. ${t.task}</b><br>
            Deadline: ${t.deadline}<br>
            Depends on: ${t.dependency || 'None'}
          </div>
        `;
      });
    }
  </script>
</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html_code)

print("✅ index.html frontend created successfully!")


✅ index.html frontend created successfully!


In [11]:
# Step 8: Serve the HTML frontend from FastAPI

from fastapi.responses import FileResponse

@app.get("/")
def serve_frontend():
    return FileResponse("index.html")


In [12]:
# Step 8: Serve the HTML frontend from FastAPI

from fastapi.responses import FileResponse

@app.get("/")
def serve_frontend():
    return FileResponse("index.html")


In [ ]:
# ✅ Step 9 (Fixed): Run FastAPI app inside Jupyter safely
import nest_asyncio
import uvicorn
import asyncio

nest_asyncio.apply()

config = uvicorn.Config(app=app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)

# Use asyncio.create_task so it runs in background
asyncio.create_task(server.serve())

print("🚀 Smart Task Planner is running at http://127.0.0.1:8000")


🚀 Smart Task Planner is running at http://127.0.0.1:8000


INFO:     Started server process [20032]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51362 - "OPTIONS /generate_plan HTTP/1.1" 200 OK
INFO:     127.0.0.1:51362 - "POST /generate_plan HTTP/1.1" 200 OK


C:\Users\HP\AppData\Local\Temp\ipykernel_20032\637558352.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  generated_at=dt.datetime.utcnow().isoformat() + "Z",


INFO:     127.0.0.1:51911 - "OPTIONS /generate_plan HTTP/1.1" 200 OK
INFO:     127.0.0.1:51911 - "POST /generate_plan HTTP/1.1" 200 OK
